In [ ]:
from sqlalchemy import create_engine, text

In [ ]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase demo"""
    login = 'student'
    password = 'student!'
    host = 'rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net'
    port = '6432'
    db_name = 'demo'
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo)

In [ ]:
engine = connection_yandex_cloud_demo(echo=True)

**! Просьба в ячейке ниже указать наименование вашей схемы, чтобы выполняемые запросы не пересекались между собой**

In [ ]:
SCHEMA_NAME = 'public'

### Оптимизация с помощью индекса

**Создадим таблицу для экспериментов и заполним её данными**

In [ ]:
sql = f'''
DROP TABLE IF EXISTS {SCHEMA_NAME}.ticket_flights_copy;

CREATE TABLE {SCHEMA_NAME}.ticket_flights_copy
 (LIKE bookings.ticket_flights);
 
INSERT INTO {SCHEMA_NAME}.ticket_flights_copy
SELECT * FROM bookings.ticket_flights
'''

engine.execute(sql)

**Посмотрим кол-во записей в созданной таблице**

In [ ]:
sql = f'''
 SELECT count(*)
   FROM {SCHEMA_NAME}.ticket_flights_copy
'''

engine.execute(sql).fetchall()[0][0]

**Сделаем запрос на поиск данных по перелету с идентификатором 130525**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525
'''

engine.execute(text(sql)).fetchall()

**Попробуем использовать нечеткий поиск, посмотрим как изменлось время выполнения запроса**

In [ ]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%130525%'
'''

engine.execute(text(sql)).fetchall()

**Создадим индекс на поле bookings.ticket_flights_copy.flight_id**

In [ ]:
sql = f'''
DROP INDEX IF EXISTS ticket_flights_copy_flight_id_index;

CREATE INDEX ticket_flights_copy_flight_id_index 
          ON {SCHEMA_NAME}.ticket_flights_copy (flight_id);
'''

engine.execute(sql)

**Выполним запросы после создания индекса, сравним время выполнения с предыдущими результатом**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id = 130525
'''

engine.execute(text(sql)).fetchall()

In [ ]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.ticket_flights_copy tfc
  WHERE tfc.flight_id::text like '%130525%'
'''

engine.execute(text(sql)).fetchall()

**Создадим копию таблицы flights и выполним запрос с фильтрацией по двум атрибутам**

In [ ]:
sql = f'''
DROP TABLE IF EXISTS {SCHEMA_NAME}.flights_copy;

CREATE TABLE {SCHEMA_NAME}.flights_copy
 (LIKE bookings.flights);
 
INSERT INTO {SCHEMA_NAME}.flights_copy
SELECT * FROM bookings.flights
'''

engine.execute(sql)

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

**Создадим составной индекс**

In [ ]:
sql = f'''
DROP INDEX IF EXISTS flights_copy_airports_index;

CREATE INDEX flights_copy_airports_index 
          ON {SCHEMA_NAME}.flights_copy (departure_airport, arrival_airport);
'''

engine.execute(sql)

**Выполним этот запрос повторно**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

**Выведем только те поля, которые участвуют в индексе**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT fc.departure_airport
       ,fc.arrival_airport
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.departure_airport = 'DME'
    AND fc.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

**Допустим, нам необходимо посмотреть данные за определённую дату о совершенных полётах**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE DATE(fc.actual_departure) = '2017-09-06'
'''

engine.execute(sql).fetchall()

**Сравним время выполнения фильтрации по равенству и наравенству**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE DATE(fc.actual_departure) > '2017-09-06'
'''

engine.execute(sql).fetchall()

**Создадим индекс на колонку с датой отправления**

In [ ]:
sql = f'''
DROP INDEX IF EXISTS flights_copy_actual_departure_index;

CREATE INDEX flights_copy_actual_departure_index ON {SCHEMA_NAME}.flights_copy (actual_departure);
'''

engine.execute(sql)

**Проверим, ускорился ли наш запрос**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.actual_departure = '2017-09-06'
'''

engine.execute(sql).fetchall()

**Сравним время выполнения фильтрации по равенству и наравенству на индексе**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.actual_departure > '2017-09-06'
'''

engine.execute(sql).fetchall()

**UNION vs UNION ALL**

In [ ]:
sql = f'''
EXPLAIN ANALYZE
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'
'''

engine.execute(sql).fetchall()

In [ ]:
sql = f'''
EXPLAIN ANALYZE 
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc
  WHERE fc.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION ALL
 SELECT * 
   FROM {SCHEMA_NAME}.flights_copy fc2
  WHERE fc2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'
'''

engine.execute(sql).fetchall()

**Временные таблицы**

In [ ]:
sql = f'''
EXPLAIN ANALYSE
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
FROM {SCHEMA_NAME}.flights_copy fc
JOIN {SCHEMA_NAME}.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id 
'''

engine.execute(sql).fetchall()

**Создаём временную таблицу**

In [ ]:
sql = f'''
CREATE TEMPORARY TABLE ticket_flights_materialized_{SCHEMA_NAME} AS 
SELECT fc.*
      ,tfc.ticket_no
      ,tfc.fare_conditions 
      ,tfc.amount 
  FROM {SCHEMA_NAME}.flights_copy fc
  JOIN {SCHEMA_NAME}.ticket_flights_copy tfc ON tfc.flight_id = fc.flight_id;
'''

engine.execute(sql)

**Сделаем запрос к веременной таблице. Как изменилось время выполнения?**

In [ ]:
sql = f'''
EXPLAIN ANALYSE
 SELECT tfm.*
   FROM ticket_flights_materialized_{SCHEMA_NAME} tfm;
'''

engine.execute(sql).fetchall()